# PyCryption Research Tutorial

This notebook serves a technical introduction, demonstration, and collaboration baseline for this repository, PyCryption.


---

## 0.0 - Getting Started: Building a Basic Algorithm

In this section we'll exploring how to rapidly test new encryption algorithms without writing tedious harnesses for testing.

In [1]:
from lib.notebook import (
    algorithm,
    with_key,
    generate_key,
    AlgorithmContext,
    ComposerSession,
    quick_test,
)

# -----------------------------------------------------------------------------
# Prototype Algorithm Development
# -----------------------------------------------------------------------------
# Use decorators for logistics (key injection, context, metrics).
# Write your own experimental crypto logic in the methods.

KEY = generate_key(32)


@algorithm("XOR-Prototype")
@with_key(KEY)
class XORPrototype:
    """
    Simple XOR-based prototype for testing the framework.
    NOT cryptographically secure - just a development placeholder.
    """

    def encrypt(self, data: bytes, ctx: AlgorithmContext) -> bytes:
        # Prototype: simple repeating-key XOR
        key = ctx.key
        return bytes(b ^ key[i % len(key)] for i, b in enumerate(data))

    def decrypt(self, data: bytes, ctx: AlgorithmContext) -> bytes:
        # XOR is symmetric
        key = ctx.key
        return bytes(b ^ key[i % len(key)] for i, b in enumerate(data))


# Verify the prototype works
quick_test(XORPrototype())

Testing: XOR-Prototype
Input: b'Hello, PyCryption!'
----------------------------------------
Encrypt: <AlgorithmResult: 18 bytes, 0.021ms>
Decrypt: <AlgorithmResult: 18 bytes, 0.008ms>
Round-trip successful!


In [2]:
# -----------------------------------------------------------------------------
# Composer Session - Benchmark Prototype vs Production Algorithms
# -----------------------------------------------------------------------------
# Register your prototype alongside proven algorithms from lib/algorithms
# to compare performance and validate behavior.

from lib.notebook import wrap_aes256gcm

# Create session and register algorithms
session = ComposerSession()

# Register our prototype
session.register(XORPrototype())

# Register proven algorithm from lib/algorithms (via adapter)
session.register(wrap_aes256gcm(KEY, name="AES-256-GCM"))

print("Registered algorithms:", session.list_algorithms())

# Test all registered algorithms
print("\n=== Round-trip Tests ===")
test_results = session.test_all()
for name, passed in test_results.items():
    status = "PASS" if passed else "FAIL"
    print(f"  {name}: {status}")

Registered algorithms: ['XOR-Prototype', 'AES-256-GCM']

=== Round-trip Tests ===
  XOR-Prototype: PASS
  AES-256-GCM: PASS


In [ ]:
# -----------------------------------------------------------------------------
# Benchmark Prototype Performance
# -----------------------------------------------------------------------------

print("=== Prototype Benchmark (10KB, 50 iterations) ===\n")
comparison = session.compare(data_size=10_000, iterations=50)

print(
    f"{'Algorithm':<20} {'Encrypt (ms)':<15} {'Decrypt (ms)':<15} {'Throughput (MB/s)':<15}"
)
print("-" * 65)
for entry in comparison:
    print(
        f"{entry['algorithm']:<20} {entry['avg_encrypt_ms']:<15} {entry['avg_decrypt_ms']:<15} {entry['throughput_mbps']:<15}"
    )

=== Prototype Benchmark (10KB, 50 iterations) ===

Algorithm            Encrypt (ms)    Decrypt (ms)    Throughput (MB/s)
-----------------------------------------------------------------
AES-256-GCM          0.053           0.04            187.27         
XOR-Prototype        1.39            1.389           7.19           


In [ ]:
# -----------------------------------------------------------------------------
# Detailed Benchmarks - Scaling Analysis
# -----------------------------------------------------------------------------

print("=== Scaling Analysis ===\n")
benchmarks = session.benchmark_all(
    data_sizes=[100, 1_000, 10_000, 100_000], iterations=20
)

for algo_name, results in benchmarks.items():
    print(f"{algo_name}:")
    print(
        f"  {'Size':<12} {'Encrypt (ms)':<15} {'Decrypt (ms)':<15} {'Throughput':<15}"
    )
    print(f"  {'-'*55}")
    for bench in results["benchmarks"]:
        size_label = f"{bench['size_bytes']:,} B"
        print(
            f"  {size_label:<12} {bench['avg_encrypt_ms']:<15} {bench['avg_decrypt_ms']:<15} {bench['throughput_mbps']:<15} MB/s"
        )

=== Scaling Analysis ===

XOR-Prototype:
  Size         Encrypt (ms)    Decrypt (ms)    Throughput     
  -------------------------------------------------------
  100 B        0.039           0.031           2.55            MB/s
  1,000 B      0.25            0.246           4.0             MB/s
  10,000 B     1.354           1.309           7.39            MB/s
  100,000 B    12.635          12.859          7.91            MB/s
AES-256-GCM:
  Size         Encrypt (ms)    Decrypt (ms)    Throughput     
  -------------------------------------------------------
  100 B        0.023           0.016           4.32            MB/s
  1,000 B      0.018           0.016           55.87           MB/s
  10,000 B     0.02            0.018           493.83          MB/s
  100,000 B    0.039           0.037           2560.82         MB/s


In [ ]:
# -----------------------------------------------------------------------------
# Session Metrics - Aggregated Statistics
# -----------------------------------------------------------------------------

print("=== Session Report ===\n")
report = session.report()

for algo_name, metrics in report.items():
    print(f"{algo_name}:")
    print(
        f"  Operations: {metrics['encrypt_calls']} encrypt, {metrics['decrypt_calls']} decrypt"
    )
    print(
        f"  Avg timing: {metrics['avg_encrypt_ms']:.3f}ms encrypt, {metrics['avg_decrypt_ms']:.3f}ms decrypt"
    )
    print(f"  Total data: {metrics['total_bytes_processed']:,} bytes")
    if metrics["errors"] > 0:
        print(f"  Errors: {metrics['errors']}")
    print()

=== Session Report ===

XOR-Prototype:
  Operations: 1 encrypt, 1 decrypt
  Avg timing: 0.021ms encrypt, 0.007ms decrypt
  Total data: 36 bytes

AES-256-GCM:
  Operations: 1 encrypt, 1 decrypt
  Avg timing: 2.110ms encrypt, 0.028ms decrypt
  Total data: 52 bytes



---

## 1.0 Composing Secure Algorithms

Beyond a simple byte emitter, working with
